In general, there are three types of constraints:
- Constraint repair
- Penalty methods
- Feasibility-preserving operations

From an implementation standpoint, constraint repair takes the updated particles at time $t$ and applies some function that modifies the particle positions to be enforce feasibility. Penalty methods instead modify the computed objective score after the particle position update as a function of the new positions and the constraints. Finally, feasibility-preserving operations overload the algebraic operations (addition and multiplication) to guarantee feasibility (e.g., using the Aitchison geometry to enforce sum-to-one constraints). 

In [129]:
include("./src/MetaDoE.jl")
using .MetaDoE: Experiments, ConstraintEnforcement, PSO, Objectives, Designs
using LinearAlgebra
using NPZ

In [130]:
function unit_hypercube_constraints(d::Int)
    I_d = Matrix{Float64}(I, d, d)
    A = vcat(I_d, -I_d)
    b = vcat(ones(d), zeros(d))
    return A, b
end

function hypercube_constraints(n::Int, k::Real)
    I_n = Matrix{Float64}(I, n, n)
    A = vcat(I_n, -I_n)
    b = vcat(fill(Float64(k), n), fill(Float64(k), n))
    return A, b
end


hypercube_constraints (generic function with 1 method)

In [152]:
A, b = hypercube_constraints(3, 100)
experiment = Experiments.create(1, 3)
experiment = Experiments.with_linear_constraints(experiment, A, b)
experiment = Experiments.with_factor_ratio(experiment, 2, 3, 2.0)
experiment = Experiments.with_factor_ratio(experiment, 1, 3, 2.0)
A = copy(experiment.constraints.A)
b = Float64.([100, 100, 100, 100, 100, 100, 0, 0])

8-element Vector{Float64}:
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
   0.0
   0.0

In [132]:
experiment = Experiments.create(1, 3)
experiment = Experiments.with_linear_constraints(experiment, A, b)

runner_params = PSO.runner_params(1000, 500, 1e-6)
pso_params = PSO.create_hyperparams(100)
context = PSO.create_context(
    experiment, 
    Objectives.griewank; 
    callback = PSO.aggregate_results(;save_world=true), 
    runner_params=runner_params
)

Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+02]
  Cost   [0e+00, 0e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No LP invertible representation for getDualRay
Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 2e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 1e+02]
Presolving model
8 rows, 4 cols, 18 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-8); columns 0(-4); elements 0(-18) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  5.7294901688e+01
HiGHS run time      :       

Main.MetaDoE.PSO.OptimizationContext(Main.MetaDoE.PSO.Swarm(Main.MetaDoE.PSO.ParticleState([3.3336697437015133; 11.12016751839908; … ; -50.363897102659635; -44.48516385543744;;; 32.26217654001104; 45.19025473063888; … ; -76.84879734669997; -79.12249200106494;;; 30.4360628587781; 52.261796008528634; … ; 66.54765596929076; 68.65703339425971], [59.97772170465628; 1.449161762970725; … ; -20.508512694899117; -45.70831754835076;;; 1.1032131934501805; -64.52746649776063; … ; -34.28006632717358; -34.94725831622862;;; 61.789642807355946; 43.69543015704352; … ; 25.36450465160826; 22.250139092901406]), sparse([1, 18, 2, 28, 73, 3, 89, 94, 4, 5  …  88, 98, 32, 52, 94, 99, 13, 40, 93, 100], [1, 1, 2, 2, 2, 3, 3, 3, 4, 5  …  98, 98, 99, 99, 99, 99, 100, 100, 100, 100], Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 100, 100), Main.MetaDoE.PSO.ParticleFitness([1.3011768172531588, 2.189884536599198, 2.457819316575816, 3.444434777197756, 3.183891265917591, 4.157157039727868, 4.338

In [133]:
runner_state, history = PSO.optimize(context)
optimizer, optimal_score = PSO.get_optimizer(runner_state)
optimal_score

0.0

In [134]:
optimizer

1×3 Matrix{Float64}:
 -9.00981e-9  -5.47236e-9  -2.73618e-9

In [135]:
function save_history(history::Vector; location = "pso_griewank.npy")
    velocities = cat([h.state.velocities for h in history]...; dims=2)
    velocities = permutedims(velocities, (2, 1, 3))

    positions = cat([h.state.positions for h in history]...; dims=2)
    positions = permutedims(positions, (2, 1, 3))

    scores = cat([h.fitness.scores for h in history]...; dims=2)'
    npzwrite(location, Dict("positions" => positions, "velocities" => velocities, "scores" => scores))
end

save_history (generic function with 1 method)

In [136]:
save_history(history)

In [137]:
using CDDLib
using Polyhedra

In [153]:
get_vertices = (p) -> collect(points(p))

p = polyhedron(hrep(A, b), CDDLib.Library())
verts = get_vertices(p)
verts = hcat(verts...)'
npzwrite("verts.npy", verts)

In [139]:
initializer = Designs.create_initializer(experiment.constraints, experiment.N, experiment.K)

Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+02]
  Cost   [0e+00, 0e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No LP invertible representation for getDualRay
Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 2e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 1e+02]
Presolving model
8 rows, 4 cols, 18 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-8); columns 0(-4); elements 0(-18) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  5.7294901688e+01
HiGHS run time      :       

(::Main.MetaDoE.Designs.var"#sample_constraints#9"{Random.TaskLocalRNG, Int64, Int64, Matrix{Float64}, Vector{Float64}, Vector{Float64}, Int64, DefaultLibrary{Float64}}) (generic function with 1 method)

In [140]:
using .MetaDoE: TensorOps

In [141]:
npzwrite("sample.npy", TensorOps.squeeze(initializer(10_000)))

In [142]:
TensorOps.squeeze(initializer(20))

20×3 Matrix{Float64}:
 -89.8242   -75.3978    -13.5421
 -94.5611   -86.4025      7.14961
 -95.2809   -85.6779      8.66902
 -92.847    -93.5142     10.2761
 -72.9712   -72.231     -17.2032
 -88.8963    15.5274     18.6999
 -93.9559    23.1825     27.0228
 -93.0755    24.4048     31.2028
 -71.8728    42.4747     77.1161
 -39.1076    32.5622     29.1638
 -75.959      3.73425    14.077
 -10.1558    -0.811757   63.0794
 -14.4355   -18.9985     37.983
 -22.3751    -2.34897    45.8154
   4.38389   -2.39669    12.9151
  10.9619    68.5332     48.659
  13.7542    50.2509     91.3139
  14.0083    50.2199     90.7709
   9.42835   42.1102     96.5315
 -10.4615    -2.15378    99.7596